## Lineær og Polynomisk Regression for Græskarpriser - Lektion 3
<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Infografik af Dasani Madipalli</figcaption>


<!--![Infografik af Dasani Madipalli](../../../../../../2-Regression/3-Linear/images/linear-polynomial.png){width="800"}-->

#### Introduktion

Indtil videre har du udforsket, hvad regression er, med eksempeldata indsamlet fra græskarpris-datasættet, som vi vil bruge gennem denne lektion. Du har også visualiseret det ved hjælp af `ggplot2`.💪

Nu er du klar til at dykke dybere ned i regression for ML. I denne lektion vil du lære mere om to typer regression: *grundlæggende lineær regression* og *polynomisk regression*, sammen med noget af den matematik, der ligger til grund for disse teknikker.

> Gennem hele dette pensum antager vi minimal viden om matematik og søger at gøre det tilgængeligt for studerende fra andre områder, så hold øje med noter, 🧮 fremhævelser, diagrammer og andre læringsværktøjer, der kan hjælpe med forståelsen.

#### Forberedelse

Som en påmindelse, du indlæser disse data for at kunne stille spørgsmål til dem.

-   Hvornår er det bedste tidspunkt at købe græskar?

-   Hvilken pris kan jeg forvente for en kasse med miniaturegræskar?

-   Skal jeg købe dem i halv-bushel kurve eller i 1 1/9 bushel kasser? Lad os fortsætte med at grave i disse data.

I den forrige lektion oprettede du en `tibble` (en moderne genfortolkning af dataframes) og fyldte den med en del af det originale datasæt, hvor du standardiserede prisen pr. bushel. Ved at gøre det var du dog kun i stand til at samle omkring 400 datapunkter, og kun for efterårsmånederne. Måske kan vi få lidt mere detaljeret indsigt i dataene ved at rense dem yderligere? Det må vi se... 🕵️‍♀️

Til denne opgave skal vi bruge følgende pakker:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) er en [samling af R-pakker](https://www.tidyverse.org/packages), der er designet til at gøre datavidenskab hurtigere, lettere og sjovere!

-   `tidymodels`: [tidymodels](https://www.tidymodels.org/) er en [rammeværk af pakker](https://www.tidymodels.org/packages/) til modellering og maskinlæring.

-   `janitor`: [janitor-pakken](https://github.com/sfirke/janitor) tilbyder enkle værktøjer til at undersøge og rense beskidte data.

-   `corrplot`: [corrplot-pakken](https://cran.r-project.org/web/packages/corrplot/vignettes/corrplot-intro.html) giver et visuelt udforskningsværktøj til korrelationsmatricer, der understøtter automatisk variabelomorganisering for at hjælpe med at opdage skjulte mønstre blandt variabler.

Du kan installere dem som:

`install.packages(c("tidyverse", "tidymodels", "janitor", "corrplot"))`

Scriptet nedenfor tjekker, om du har de nødvendige pakker til at fuldføre dette modul og installerer dem for dig, hvis de mangler.


In [ ]:
suppressWarnings(if (!require("pacman")) install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, janitor, corrplot)

Vi vil senere indlæse disse fantastiske pakker og gøre dem tilgængelige i vores nuværende R-session. (Dette er kun for illustration, `pacman::p_load()` har allerede gjort det for dig)

## 1. En lineær regressionslinje

Som du lærte i Lektion 1, er målet med en lineær regressionsøvelse at kunne tegne en *linje* *af* *bedste tilpasning* for at:

-   **Vis variableforhold**. Vise forholdet mellem variabler.

-   **Foretage forudsigelser**. Foretage præcise forudsigelser om, hvor et nyt datapunkt vil falde i forhold til den linje.

For at tegne denne type linje bruger vi en statistisk teknik kaldet **Mindste Kvadraters Regression**. Udtrykket `mindste kvadrater` betyder, at alle datapunkterne omkring regressionslinjen kvadreres og derefter lægges sammen. Ideelt set er den endelige sum så lille som muligt, fordi vi ønsker et lavt antal fejl, eller `mindste kvadrater`. Som sådan er linjen med den bedste tilpasning den linje, der giver os den laveste værdi for summen af de kvadrerede fejl - deraf navnet *mindste kvadraters regression*.

Vi gør dette, fordi vi ønsker at modellere en linje, der har den mindste kumulative afstand fra alle vores datapunkter. Vi kvadrerer også termerne, før vi lægger dem sammen, da vi er interesserede i deres størrelse snarere end deres retning.

> **🧮 Vis mig matematikken**
>
> Denne linje, kaldet *linjen med den bedste tilpasning*, kan udtrykkes ved [en ligning](https://en.wikipedia.org/wiki/Simple_linear_regression):
>
>     Y = a + bX
>
> `X` er '`forklarende variabel` eller `prediktor`'. `Y` er '`afhængig variabel` eller `resultat`'. Hældningen af linjen er `b`, og `a` er skæringspunktet med y-aksen, som refererer til værdien af `Y`, når `X = 0`.
>

> ![](../../../../../../2-Regression/3-Linear/solution/images/slope.png "hældning = $y/x$")
    Infografik af Jen Looper
>
> Først beregnes hældningen `b`.
>
> Med andre ord, og med henvisning til vores græskardatas oprindelige spørgsmål: "forudsige prisen på et græskar pr. tønde efter måned", ville `X` referere til prisen, og `Y` ville referere til salgsdatoen.
>
> ![](../../../../../../2-Regression/3-Linear/solution/images/calculation.png)
    Infografik af Jen Looper
> 
> Beregn værdien af Y. Hvis du betaler omkring \$4, må det være april!
>
> Matematikken, der beregner linjen, skal demonstrere hældningen af linjen, som også afhænger af skæringspunktet, eller hvor `Y` er placeret, når `X = 0`.
>
> Du kan se metoden til beregning af disse værdier på webstedet [Math is Fun](https://www.mathsisfun.com/data/least-squares-regression.html). Besøg også [denne Mindste Kvadraters beregner](https://www.mathsisfun.com/data/least-squares-calculator.html) for at se, hvordan tallenes værdier påvirker linjen.

Ikke så skræmmende, vel? 🤓

#### Korrelation

Et andet begreb, du skal forstå, er **Korrelationskoefficienten** mellem givne X- og Y-variabler. Ved hjælp af et scatterplot kan du hurtigt visualisere denne koefficient. Et plot med datapunkter spredt i en pæn linje har høj korrelation, men et plot med datapunkter spredt overalt mellem X og Y har lav korrelation.

En god lineær regressionsmodel vil være en, der har en høj (nærmere 1 end 0) korrelationskoefficient ved brug af Mindste Kvadraters Regression-metoden med en regressionslinje.


## **2. En dans med data: oprettelse af en data frame til modellering**

<p >
   <img src="../../images/janitor.jpg"
   width="700"/>
   <figcaption>Kunstværk af @allison_horst</figcaption>


<!--![Kunstværk af \@allison_horst](../../../../../../2-Regression/3-Linear/images/janitor.jpg){width="700"}-->


Indlæs de nødvendige biblioteker og datasæt. Konverter dataene til en data frame, der indeholder et udsnit af dataene:

-   Kun få græskar, der er prissat pr. tønde

-   Konverter datoen til en måned

-   Beregn prisen som et gennemsnit af høj og lav pris

-   Konverter prisen, så den afspejler prissætningen pr. tønde

> Vi dækkede disse trin i [den forrige lektion](https://github.com/microsoft/ML-For-Beginners/blob/main/2-Regression/2-Data/solution/lesson_2-R.ipynb).


In [ ]:
# Load the core Tidyverse packages
library(tidyverse)
library(lubridate)

# Import the pumpkins data
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv")


# Get a glimpse and dimensions of the data
glimpse(pumpkins)


# Print the first 50 rows of the data set
pumpkins %>% 
  slice_head(n = 5)

I eventyrets ånd, lad os udforske [`janitor-pakken`](../../../../../../2-Regression/3-Linear/solution/R/github.com/sfirke/janitor), der tilbyder simple funktioner til at undersøge og rense beskidte data. For eksempel, lad os tage et kig på kolonnenavnene for vores data:


In [ ]:
# Return column names
pumpkins %>% 
  names()

🤔 Vi kan gøre det bedre. Lad os gøre disse kolonnenavne `friendR` ved at konvertere dem til [snake_case](https://en.wikipedia.org/wiki/Snake_case)-konventionen ved hjælp af `janitor::clean_names`. For at finde ud af mere om denne funktion: `?clean_names`


In [ ]:
# Clean names to the snake_case convention
pumpkins <- pumpkins %>% 
  clean_names(case = "snake")

# Return column names
pumpkins %>% 
  names()

Meget tidyR 🧹! Nu, en dans med dataene ved hjælp af `dplyr` som i den tidligere lektion! 💃


In [ ]:
# Select desired columns
pumpkins <- pumpkins %>% 
  select(variety, city_name, package, low_price, high_price, date)



# Extract the month from the dates to a new column
pumpkins <- pumpkins %>%
  mutate(date = mdy(date),
         month = month(date)) %>% 
  select(-date)



# Create a new column for average Price
pumpkins <- pumpkins %>% 
  mutate(price = (low_price + high_price)/2)


# Retain only pumpkins with the string "bushel"
new_pumpkins <- pumpkins %>% 
  filter(str_detect(string = package, pattern = "bushel"))


# Normalize the pricing so that you show the pricing per bushel, not per 1 1/9 or 1/2 bushel
new_pumpkins <- new_pumpkins %>% 
  mutate(price = case_when(
    str_detect(package, "1 1/9") ~ price/(1.1),
    str_detect(package, "1/2") ~ price*2,
    TRUE ~ price))

# Relocate column positions
new_pumpkins <- new_pumpkins %>% 
  relocate(month, .before = variety)


# Display the first 5 rows
new_pumpkins %>% 
  slice_head(n = 5)

Godt klaret!👌 Du har nu et rent og ryddeligt datasæt, som du kan bruge til at bygge din nye regressionsmodel!

Hvad siger du til et scatterplot?


In [ ]:
# Set theme
theme_set(theme_light())

# Make a scatter plot of month and price
new_pumpkins %>% 
  ggplot(mapping = aes(x = month, y = price)) +
  geom_point(size = 1.6)


Et spredningsdiagram minder os om, at vi kun har månedsdata fra august til december. Vi har sandsynligvis brug for mere data for at kunne drage konklusioner på en lineær måde.

Lad os tage et kig på vores modelleringsdata igen:


In [ ]:
# Display first 5 rows
new_pumpkins %>% 
  slice_head(n = 5)

Hvad hvis vi ville forudsige `price` på et græskar baseret på `city` eller `package` kolonnerne, som er af typen karakter? Eller endnu enklere, hvordan kunne vi finde korrelationen (som kræver, at begge input er numeriske) mellem for eksempel `package` og `price`? 🤷🤷

Maskinlæringsmodeller fungerer bedst med numeriske egenskaber frem for tekstværdier, så det er generelt nødvendigt at konvertere kategoriske egenskaber til numeriske repræsentationer.

Det betyder, at vi skal finde en måde at omformatere vores prediktorer på, så de bliver nemmere for en model at bruge effektivt, en proces kendt som `feature engineering`.


## 3. Forbehandling af data til modellering med opskrifter 👩‍🍳👨‍🍳

Aktiviteter, der omformaterer prædiktorværdier for at gøre dem lettere for en model at bruge effektivt, kaldes `feature engineering`.

Forskellige modeller har forskellige krav til forbehandling. For eksempel kræver mindste kvadrater `kodning af kategoriske variabler` såsom måned, sort og bynavn. Dette indebærer simpelthen at `oversætte` en kolonne med `kategoriske værdier` til en eller flere `numeriske kolonner`, der erstatter den oprindelige.

For eksempel, antag at dine data indeholder følgende kategoriske variabel:

|   by    |
|:-------:|
| Denver  |
| Nairobi |
|  Tokyo  |

Du kan anvende *ordinal kodning* for at erstatte hver kategori med en unik heltalsværdi, som dette:

| by  |
|:---:|
|  0  |
|  1  |
|  2  |

Og det er præcis, hvad vi vil gøre med vores data!

I denne sektion vil vi udforske endnu en fantastisk Tidymodels-pakke: [recipes](https://tidymodels.github.io/recipes/) - som er designet til at hjælpe dig med at forbehandle dine data **før** du træner din model. I sin kerne er en opskrift et objekt, der definerer, hvilke trin der skal anvendes på et datasæt for at gøre det klar til modellering.

Lad os nu oprette en opskrift, der forbereder vores data til modellering ved at erstatte en unik heltalsværdi for alle observationer i prædiktorkolonnerne:


In [ ]:
# Specify a recipe
pumpkins_recipe <- recipe(price ~ ., data = new_pumpkins) %>% 
  step_integer(all_predictors(), zero_based = TRUE)


# Print out the recipe
pumpkins_recipe

Fantastisk! 👏 Vi har lige oprettet vores første opskrift, der specificerer et resultat (pris) og de tilsvarende prediktorer, og som sørger for, at alle prediktorkolonner bliver kodet som et sæt heltal 🙌! Lad os hurtigt bryde det ned:

-   Kaldet til `recipe()` med en formel fortæller opskriften *rollerne* for variablerne ved at bruge `new_pumpkins`-dataene som reference. For eksempel er `price`-kolonnen blevet tildelt rollen som `outcome`, mens resten af kolonnerne er blevet tildelt rollen som `predictor`.

-   `step_integer(all_predictors(), zero_based = TRUE)` specificerer, at alle prediktorer skal konverteres til et sæt heltal, hvor nummereringen starter ved 0.

Vi er sikre på, at du måske tænker: "Det her er så fedt!! Men hvad nu hvis jeg havde brug for at bekræfte, at opskrifterne gør præcis det, jeg forventer? 🤔"

Det er en fantastisk tanke! Ser du, når din opskrift er defineret, kan du estimere de parametre, der kræves for faktisk at forbehandle dataene, og derefter udtrække de forarbejdede data. Normalt behøver du ikke gøre dette, når du bruger Tidymodels (vi ser den normale konvention om et øjeblik -> `workflows`), men det kan være nyttigt, hvis du vil lave en slags sanity check for at bekræfte, at opskrifterne gør, hvad du forventer.

Til det skal du bruge to ekstra verber: `prep()` og `bake()`, og som altid hjælper vores små R-venner fra [`Allison Horst`](https://github.com/allisonhorst/stats-illustrations) dig med at forstå dette bedre!

<p >
   <img src="../../images/recipes.png"
   width="550"/>
   <figcaption>Kunstværk af @allison_horst</figcaption>


[`prep()`](https://recipes.tidymodels.org/reference/prep.html): estimerer de nødvendige parametre fra et træningssæt, som senere kan anvendes på andre datasæt. For eksempel, for en given predictor-kolonne, hvilken observation vil blive tildelt heltal 0, 1, 2 osv.

[`bake()`](https://recipes.tidymodels.org/reference/bake.html): tager en forberedt opskrift og anvender operationerne på ethvert datasæt.

Når det er sagt, lad os forberede og anvende vores opskrifter for virkelig at bekræfte, at under overfladen vil predictor-kolonnerne først blive kodet, før en model tilpasses.


In [ ]:
# Prep the recipe
pumpkins_prep <- prep(pumpkins_recipe)

# Bake the recipe to extract a preprocessed new_pumpkins data
baked_pumpkins <- bake(pumpkins_prep, new_data = NULL)

# Print out the baked data set
baked_pumpkins %>% 
  slice_head(n = 10)

Woo-hoo!🥳 De behandlede data `baked_pumpkins` har alle sine prædiktorer kodet, hvilket bekræfter, at de foruddefinerede forbehandlingsskridt i vores opskrift fungerer som forventet. Dette gør det sværere for dig at læse, men meget mere forståeligt for Tidymodels! Tag dig tid til at finde ud af, hvilken observation der er blevet kortlagt til et tilsvarende heltal.

Det er også værd at nævne, at `baked_pumpkins` er en data frame, som vi kan udføre beregninger på.

For eksempel, lad os prøve at finde en god korrelation mellem to punkter i dine data for potentielt at bygge en god prædiktiv model. Vi bruger funktionen `cor()` til dette. Skriv `?cor()` for at finde ud af mere om funktionen.


In [ ]:
# Find the correlation between the city_name and the price
cor(baked_pumpkins$city_name, baked_pumpkins$price)

# Find the correlation between the package and the price
cor(baked_pumpkins$package, baked_pumpkins$price)


Som det viser sig, er der kun en svag sammenhæng mellem By og Pris. Derimod er der en lidt bedre sammenhæng mellem Pakke og dens Pris. Det giver mening, ikke? Normalt, jo større produktkassen er, desto højere er prisen.

Mens vi er i gang, lad os også prøve at visualisere en korrelationsmatrix for alle kolonnerne ved hjælp af `corrplot`-pakken.


In [ ]:
# Load the corrplot package
library(corrplot)

# Obtain correlation matrix
corr_mat <- cor(baked_pumpkins %>% 
                  # Drop columns that are not really informative
                  select(-c(low_price, high_price)))

# Make a correlation plot between the variables
corrplot(corr_mat, method = "shade", shade.col = NA, tl.col = "black", tl.srt = 45, addCoef.col = "black", cl.pos = "n", order = "original")

🤩🤩 Meget bedre.

Et godt spørgsmål at stille til disse data kunne være: '`Hvilken pris kan jeg forvente for en given græskarpakke?`' Lad os komme i gang!

> Note: Når du **`bake()`** den forberedte opskrift **`pumpkins_prep`** med **`new_data = NULL`**, udtrækker du de behandlede (dvs. kodede) træningsdata. Hvis du havde et andet datasæt, for eksempel et testdatasæt, og gerne ville se, hvordan en opskrift ville forbehandle det, skulle du blot bage **`pumpkins_prep`** med **`new_data = test_set`**

## 4. Byg en lineær regressionsmodel

<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Infografik af Dasani Madipalli</figcaption>


<!--![Infografik af Dasani Madipalli](../../../../../../2-Regression/3-Linear/images/linear-polynomial.png){width="800"}-->


Nu hvor vi har opbygget en opskrift og faktisk bekræftet, at dataene vil blive forbehandlet korrekt, lad os nu opbygge en regressionsmodel for at besvare spørgsmålet: `Hvilken pris kan jeg forvente for en given græskarpakke?`

#### Træn en lineær regressionsmodel ved hjælp af træningssættet

Som du måske allerede har regnet ud, er kolonnen *price* den `afhængige` variabel, mens kolonnen *package* er den `uafhængige` variabel.

For at gøre dette vil vi først opdele dataene, så 80% går til træningssættet og 20% til test-sættet, derefter definere en opskrift, der vil kode den uafhængige variabel som et sæt heltal, og derefter opbygge en modelspecifikation. Vi vil ikke forberede og bage vores opskrift, da vi allerede ved, at den vil forbehandle dataene som forventet.


In [ ]:
set.seed(2056)
# Split the data into training and test sets
pumpkins_split <- new_pumpkins %>% 
  initial_split(prop = 0.8)


# Extract training and test data
pumpkins_train <- training(pumpkins_split)
pumpkins_test <- testing(pumpkins_split)



# Create a recipe for preprocessing the data
lm_pumpkins_recipe <- recipe(price ~ package, data = pumpkins_train) %>% 
  step_integer(all_predictors(), zero_based = TRUE)



# Create a linear model specification
lm_spec <- linear_reg() %>% 
  set_engine("lm") %>% 
  set_mode("regression")

Godt klaret! Nu hvor vi har en opskrift og en modelspecifikation, skal vi finde en måde at samle dem i et objekt, der først forbehandler dataene (prep+bake bag kulisserne), tilpasser modellen til de forbehandlede data og også giver mulighed for potentielle efterbehandlingsaktiviteter. Hvordan lyder det for din ro i sindet!🤩

I Tidymodels kaldes dette praktiske objekt en [`workflow`](https://workflows.tidymodels.org/) og indeholder bekvemt dine modelkomponenter! Det er det, vi ville kalde *pipelines* i *Python*.

Så lad os samle det hele i en workflow!📦


In [ ]:
# Hold modelling components in a workflow
lm_wf <- workflow() %>% 
  add_recipe(lm_pumpkins_recipe) %>% 
  add_model(lm_spec)

# Print out the workflow
lm_wf

👌 Som en ekstra bonus kan en arbejdsproces tilpasses/trænes på næsten samme måde som en model kan.


In [ ]:
# Train the model
lm_wf_fit <- lm_wf %>% 
  fit(data = pumpkins_train)

# Print the model coefficients learned 
lm_wf_fit

Fra modeloutputtet kan vi se de koefficienter, der blev lært under træningen. De repræsenterer koefficienterne for den bedste tilpasningslinje, som giver os den laveste samlede fejl mellem den faktiske og den forudsagte variabel.

#### Evaluér modellens præstation ved hjælp af testdatasættet

Det er tid til at se, hvordan modellen klarede sig 📏! Hvordan gør vi det?

Nu hvor vi har trænet modellen, kan vi bruge den til at lave forudsigelser for test_sættet ved hjælp af `parsnip::predict()`. Derefter kan vi sammenligne disse forudsigelser med de faktiske mærkeværdier for at vurdere, hvor godt (eller dårligt!) modellen fungerer.

Lad os starte med at lave forudsigelser for testdatasættet og derefter binde kolonnerne til testdatasættet.


In [ ]:
# Make predictions for the test set
predictions <- lm_wf_fit %>% 
  predict(new_data = pumpkins_test)


# Bind predictions to the test set
lm_results <- pumpkins_test %>% 
  select(c(package, price)) %>% 
  bind_cols(predictions)


# Print the first ten rows of the tibble
lm_results %>% 
  slice_head(n = 10)

Ja, du har lige trænet en model og brugt den til at lave forudsigelser!🔮 Er den god? Lad os evaluere modellens præstation!

I Tidymodels gør vi dette ved hjælp af `yardstick::metrics()`! For lineær regression fokuserer vi på følgende metrikker:

-   `Root Mean Square Error (RMSE)`: Kvadratroden af [MSE](https://en.wikipedia.org/wiki/Mean_squared_error). Dette giver en absolut metrik i samme enhed som labelen (i dette tilfælde prisen på et græskar). Jo mindre værdien er, desto bedre er modellen (forenklet sagt repræsenterer det den gennemsnitlige pris, som forudsigelserne er forkerte med!)

-   `Coefficient of Determination (normalt kendt som R-squared eller R2)`: En relativ metrik, hvor en højere værdi betyder en bedre modeltilpasning. Grundlæggende repræsenterer denne metrik, hvor meget af variansen mellem de forudsagte og faktiske labelværdier modellen er i stand til at forklare.


In [ ]:
# Evaluate performance of linear regression
metrics(data = lm_results,
        truth = price,
        estimate = .pred)

Der går modelpræstationen. Lad os se, om vi kan få en bedre indikation ved at visualisere et spredningsdiagram af pakken og prisen og derefter bruge de forudsigelser, der er lavet, til at overlejre en linje med bedste tilpasning.

Det betyder, at vi skal forberede og bearbejde testdatasættet for at kode pakke-kolonnen og derefter binde dette til de forudsigelser, som vores model har lavet.


In [ ]:
# Encode package column
package_encode <- lm_pumpkins_recipe %>% 
  prep() %>% 
  bake(new_data = pumpkins_test) %>% 
  select(package)


# Bind encoded package column to the results
lm_results <- lm_results %>% 
  bind_cols(package_encode %>% 
              rename(package_integer = package)) %>% 
  relocate(package_integer, .after = package)


# Print new results data frame
lm_results %>% 
  slice_head(n = 5)


# Make a scatter plot
lm_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_line(aes(y = .pred), color = "orange", size = 1.2) +
  xlab("package")
  


Som du kan se, generaliserer lineær regressionsmodellen ikke særlig godt forholdet mellem en pakke og dens tilsvarende pris.

🎃 Tillykke, du har lige skabt en model, der kan hjælpe med at forudsige prisen på nogle få græskarsorter. Din græskarhave til højtiden bliver smuk. Men du kan sandsynligvis lave en bedre model!

## 5. Byg en polynomisk regressionsmodel

<p >
   <img src="../../images/linear-polynomial.png"
   width="800"/>
   <figcaption>Infografik af Dasani Madipalli</figcaption>


<!--![Infografik af Dasani Madipalli](../../../../../../2-Regression/3-Linear/images/linear-polynomial.png){width="800"}-->


Nogle gange har vores data måske ikke en lineær sammenhæng, men vi vil stadig gerne forudsige et resultat. Polynomisk regression kan hjælpe os med at lave forudsigelser for mere komplekse ikke-lineære sammenhænge.

Tag for eksempel forholdet mellem pakke og pris i vores græskar-datasæt. Selvom der nogle gange er en lineær sammenhæng mellem variablerne - jo større græskar i volumen, jo højere pris - kan disse sammenhænge nogle gange ikke afbildes som et plan eller en lige linje.

> ✅ Her er [nogle flere eksempler](https://online.stat.psu.edu/stat501/lesson/9/9.8) på data, der kunne bruge polynomisk regression
>
> Tag et nyt kig på forholdet mellem Sort og Pris i det tidligere plot. Ser dette scatterplot ud som om det nødvendigvis skal analyseres med en lige linje? Måske ikke. I dette tilfælde kan du prøve polynomisk regression.
>
> ✅ Polynomier er matematiske udtryk, der kan bestå af en eller flere variabler og koefficienter

#### Træn en polynomisk regressionsmodel ved hjælp af træningssættet

Polynomisk regression skaber en *kurvet linje* for bedre at tilpasse sig ikke-lineære data.

Lad os se, om en polynomisk model vil klare sig bedre til at lave forudsigelser. Vi følger en nogenlunde lignende procedure som før:

-   Opret en opskrift, der angiver de forbehandlingsskridt, der skal udføres på vores data for at gøre dem klar til modellering, dvs.: kodning af prædiktorer og beregning af polynomier af grad *n*

-   Byg en modelspecifikation

-   Kombiner opskriften og modelspecifikationen i en arbejdsgang

-   Opret en model ved at tilpasse arbejdsgangen

-   Evaluer, hvor godt modellen klarer sig på testdataene

Lad os komme i gang!


In [ ]:
# Specify a recipe
poly_pumpkins_recipe <-
  recipe(price ~ package, data = pumpkins_train) %>%
  step_integer(all_predictors(), zero_based = TRUE) %>% 
  step_poly(all_predictors(), degree = 4)


# Create a model specification
poly_spec <- linear_reg() %>% 
  set_engine("lm") %>% 
  set_mode("regression")


# Bundle recipe and model spec into a workflow
poly_wf <- workflow() %>% 
  add_recipe(poly_pumpkins_recipe) %>% 
  add_model(poly_spec)


# Create a model
poly_wf_fit <- poly_wf %>% 
  fit(data = pumpkins_train)


# Print learned model coefficients
poly_wf_fit

  

#### Evaluér modellens ydeevne

👏👏Du har bygget en polynomisk model, lad os lave forudsigelser på testdatasættet!


In [ ]:
# Make price predictions on test data
poly_results <- poly_wf_fit %>% predict(new_data = pumpkins_test) %>% 
  bind_cols(pumpkins_test %>% select(c(package, price))) %>% 
  relocate(.pred, .after = last_col())


# Print the results
poly_results %>% 
  slice_head(n = 10)

Woo-hoo, lad os evaluere, hvordan modellen klarede sig på test_sættet ved hjælp af `yardstick::metrics()`.


In [ ]:
metrics(data = poly_results, truth = price, estimate = .pred)

🤩🤩 Meget bedre ydeevne.

`rmse` faldt fra omkring 7 til omkring 3, hvilket indikerer en reduceret fejl mellem den faktiske pris og den forudsagte pris. Du kan *løst* tolke dette som, at forkerte forudsigelser i gennemsnit er omkring \$3 forkerte. `rsq` steg fra omkring 0,4 til 0,8.

Alle disse målinger viser, at den polynomiske model klarer sig langt bedre end den lineære model. Godt arbejde!

Lad os se, om vi kan visualisere dette!


In [ ]:
# Bind encoded package column to the results
poly_results <- poly_results %>% 
  bind_cols(package_encode %>% 
              rename(package_integer = package)) %>% 
  relocate(package_integer, .after = package)


# Print new results data frame
poly_results %>% 
  slice_head(n = 5)


# Make a scatter plot
poly_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_line(aes(y = .pred), color = "midnightblue", size = 1.2) +
  xlab("package")


Du kan se en kurvet linje, der passer bedre til dine data! 🤩

Du kan gøre dette endnu mere glidende ved at give en polynomisk formel til `geom_smooth` sådan her:


In [ ]:
# Make a scatter plot
poly_results %>% 
  ggplot(mapping = aes(x = package_integer, y = price)) +
  geom_point(size = 1.6) +
  # Overlay a line of best fit
  geom_smooth(method = lm, formula = y ~ poly(x, degree = 4), color = "midnightblue", size = 1.2, se = FALSE) +
  xlab("package")

Ligesom en glat kurve!🤩

Sådan laver du en ny forudsigelse:


In [ ]:
# Make a hypothetical data frame
hypo_tibble <- tibble(package = "bushel baskets")

# Make predictions using linear model
lm_pred <- lm_wf_fit %>% predict(new_data = hypo_tibble)

# Make predictions using polynomial model
poly_pred <- poly_wf_fit %>% predict(new_data = hypo_tibble)

# Return predictions in a list
list("linear model prediction" = lm_pred, 
     "polynomial model prediction" = poly_pred)


Forudsigelsen med `polynomial model` giver mening, givet scatterplots af `price` og `package`! Og hvis dette er en bedre model end den tidligere, når man ser på de samme data, skal du budgettere for disse dyrere græskar!

🏆 Godt klaret! Du har oprettet to regressionsmodeller i én lektion. I den sidste sektion om regression vil du lære om logistisk regression for at bestemme kategorier.

## **🚀Udfordring**

Test flere forskellige variabler i denne notebook for at se, hvordan korrelation svarer til modelnøjagtighed.

## [**Quiz efter lektionen**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/14/)

## **Gennemgang & Selvstudie**

I denne lektion lærte vi om Lineær Regression. Der findes andre vigtige typer af Regression. Læs om Stepwise, Ridge, Lasso og Elasticnet teknikker. Et godt kursus at studere for at lære mere er [Stanford Statistical Learning course](https://online.stanford.edu/courses/sohs-ystatslearning-statistical-learning).

Hvis du vil lære mere om, hvordan du bruger det fantastiske Tidymodels framework, kan du tjekke følgende ressourcer:

-   Tidymodels hjemmeside: [Kom i gang med Tidymodels](https://www.tidymodels.org/start/)

-   Max Kuhn og Julia Silge, [*Tidy Modeling with R*](https://www.tmwr.org/)*.*

###### **TAK TIL:**

[Allison Horst](https://twitter.com/allison_horst?lang=en) for at skabe de fantastiske illustrationer, der gør R mere indbydende og engagerende. Find flere illustrationer i hendes [galleri](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).



---

**Ansvarsfraskrivelse**:  
Dette dokument er blevet oversat ved hjælp af AI-oversættelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selvom vi bestræber os på nøjagtighed, skal du være opmærksom på, at automatiserede oversættelser kan indeholde fejl eller unøjagtigheder. Det originale dokument på dets oprindelige sprog bør betragtes som den autoritative kilde. For kritisk information anbefales professionel menneskelig oversættelse. Vi er ikke ansvarlige for eventuelle misforståelser eller fejltolkninger, der opstår som følge af brugen af denne oversættelse.
